## Set common parameters for analysis

In [ ]:
# extents for Sydney area
bounding_box_x = (150, 151.37)
bounding_box_y = (-34.36, -32.96)

# GDA94 / NSW Lambert
crs = 'epsg:3308'
resolution = (-40, 40)

## Land Soil Capability (LSC) data

In [ ]:
import datacube
import xarray as xr

dc = datacube.Datacube(app='LSC')

ds_acid = dc.load(
    product='acid',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_acid = ds_acid.rename({'band1':'acid'})

ds_lsc_overmod = dc.load(
    product='lsc_overmod',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_lsc_overmod = ds_lsc_overmod.rename({'band1':'lsc_overmod'})

ds_mass = dc.load(
    product='mass',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_mass = ds_mass.rename({'band1':'mass'})

ds_salinity = dc.load(
    product='salinity',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_salinity = ds_salinity.rename({'band1':'salinity'})

ds_shallow_rock = dc.load(
    product='shallow_rock',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_shallow_rock = ds_shallow_rock.rename({'band1':'shallow_rock'})

ds_structure = dc.load(
    product='structure',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_structure = ds_structure.rename({'band1':'structure'})

ds_water = dc.load(
    product='water',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_water = ds_water.rename({'band1':'water'})

ds_waterlog = dc.load(
    product='waterlog',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_waterlog = ds_waterlog.rename({'band1':'waterlog'})

ds_wind_erosion = dc.load(
    product='wind_erosion',
    x=bounding_box_x,
    y=bounding_box_y, 
    output_crs=crs,
    resolution=resolution)
ds_wind_erosion = ds_wind_erosion.rename({'band1':'wind_erosion'})


ds_lsc = xr.merge([ds_acid, ds_lsc_overmod, ds_mass, ds_salinity, 
    ds_shallow_rock, ds_structure, ds_water, ds_waterlog, ds_wind_erosion])


## Configuration of map widget and output chart

In [ ]:
from datacube.utils import geometry
from IPython.display import display,clear_output
from ipyleaflet import Map, Marker
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from pandas.plotting import register_matplotlib_converters


%matplotlib inline
%config InlineBackend.close_figures=False 
register_matplotlib_converters()

plt.ioff()
# ax=plt.gca()
f, ax = plt.subplots(1, 1, figsize = (15, 4))

mapCenter = [sum(bounding_box_y)/2, sum(bounding_box_x)/2]
mapView = Map(center=mapCenter, zoom=8)
mapCrs = geometry.CRS('EPSG:4326')
marker = Marker(location=mapCenter, draggable=False, visible=False)
mapView.add_layer(marker);

outMap = widgets.Output()
outMap.append_display_data(mapView)

coordinates = widgets.Label(value='Click map to select location.')
outMap.append_display_data(coordinates)

outChart = widgets.Output(layout=widgets.Layout(height='300px', width = '100%'))
vbox=widgets.VBox(children=(outMap,outChart))
display(vbox)


def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mouseup':
        coords = kwargs.get('coordinates')
        coordinates.value = str(coords)
        marker.location = coords
        marker.visible = True
        
        pt = geometry.point(coords[1], coords[0], mapCrs)
        ptInDcCrs = pt.to_crs(ds_lsc.mass.crs)
        x, y = ptInDcCrs.points[0]
        selection = ds_lsc.sel(y=y, x=x, method='nearest')
        
        ax.clear()
        
        varnames = list(selection.keys())

        
        index = np.arange(len(varnames))
        
        values = []
        for varname in varnames:
            val = selection.get(varname).values[0]
            values.append(val)
        
        bar_width = 1.0
        rects1 = ax.bar(
            index, values, bar_width, 
            color=['#a6cee3','#1f78b4','#b2df8a','#33a02c','#fb9a99','#e31a1c','#fdbf6f','#ff7f00','#cab2d6'],
            label='Men')
        ax.set_xlabel('LSC Classification')
        ax.set_ylabel('Class')
        ax.set_ylim(bottom=0, top=8)
        ax.set_title('Land Soil Capability (LSC) classification classes')
        ax.set_xticks(index  )
        ax.set_xticklabels(varnames)

        with outChart:
            clear_output(wait=True)
            display(ax.figure)

mapView.on_interaction(handle_interaction)
